In [33]:
import re
import pandas as pd


In [34]:
# 1. Cargar los datos
df = pd.read_excel("DatosTecnicaTurboshop.xlsx", sheet_name="F1")
df.head()

,Codigo,Nombre,Aplicaciones
0,T181-1745-2,"Cadena de Distribucion, larga, doble, 94 eslab...",NaN
1,T181-1746-0,Cadena Larga Distribucion Toyota Corolla 1600C...,NaN
2,T181-2826-8,"Cadena de Distribucion, Superior, 84 eslabones...",NaN
3,T181-2825-K,Cadena Distribucion Chevrolet N300,NaN
4,T181-3979-0,"Cadena de Distribucion, 90 Eslabones, simple",NaN


In [35]:
import re

marcas = ["Toyota", "Hyundai", "Nissan", "Chevrolet", "Suzuki", "Kia", "Ssangyong"]
modelos_compuestos = ["Santa Fe", "Grand i10", "New Actyon"]

modelos_compuestos_set = set(m.lower() for m in modelos_compuestos)
palabras_basura = {'Cadena', 'Distribucion', 'Superior', 'Corta', 'Larga', 'Simple',
                   'Bomba', 'Aceite', 'Sensor', 'Union', 'Eslabones', 'Distribución', 'De'}

def expandir_rango_años(rango):
    """
    Expande un rango de años en años individuales con mejor manejo de formatos
    Ejemplos:
    "2015-2017" → [2015, 2016, 2017]
    "15-18" → [2015, 2016, 2017, 2018]
    "2015" → [2015]
    """
    if not rango:
        return []
    
    # Limpiar el rango de espacios y caracteres extraños
    rango = re.sub(r'[^\d\-]', '', rango.strip())
    
    if '-' in rango:
        partes = rango.split('-')
        if len(partes) != 2:
            return []
        
        inicio, fin = partes
        
        # Manejar años con 2 dígitos
        if len(inicio) == 2:
            inicio = f"20{inicio}" if int(inicio) <= 30 else f"19{inicio}"
        if len(fin) == 2:
            # El año final usa el mismo siglo que el inicio si es de 2 dígitos
            siglo = inicio[:2] if len(inicio) == 4 else "20"
            fin = f"{siglo}{fin}"
        
        try:
            inicio_int = int(inicio)
            fin_int = int(fin)
            
            # Validar rango cronológico
            if inicio_int > fin_int:
                return []
            
            # Validar años razonables (entre 1980 y 2030)
            if inicio_int < 1980 or fin_int > 2030:
                return []
            
            return list(range(inicio_int, fin_int + 1))
        except ValueError:
            return []
    else:
        # Manejar año individual
        if len(rango) == 2:
            rango = f"20{rango}" if int(rango) <= 30 else f"19{rango}"
        
        try:
            año = int(rango)
            return [año] if 1980 <= año <= 2030 else []
        except ValueError:
            return []

In [36]:
def extraer_y_expandir_años(texto):
    """
    Extrae y expande rangos de años del texto con mejor manejo de contextos
    """
    # Patrones mejorados para evitar falsos positivos
    patrones = [
        r'(?<!\d)(20\d{2})\s*(?:a|l|\/|\||\\)\s*(20\d{2})(?!\d)',  # 2015 a 2020
        r'(?<!\d)(20\d{2})\s*-\s*(20\d{2})(?!\d)',  # 2015-2020
        r'(?<!\d)(20\d{2})\s*-\s*(\d{2})(?!\d)',    # 2015-20
        r'(?<!\d)(\d{2})\s*-\s*(20\d{2})(?!\d)',    # 15-2020
        r'(?<!\d)(\d{2})\s*-\s*(\d{2})(?!\d)',      # 15-20
        r'(?<!\d)(20\d{2})(?!\d)',                  # 2020 solo
        r'(?<!\d)(9[0-9]|0[0-9])(?!\d)'             # 90-99 o 00-09 (solo si no están junto a otros números)
    ]
    
    años_encontrados = []
    
    for patron in patrones:
        matches = re.finditer(patron, texto)
        for match in matches:
            grupos = match.groups()
            
            if len(grupos) > 1:  # Es un rango
                inicio, fin = grupos[0], grupos[1]
                rango = f"{inicio}-{fin}"
            else:  # Es un año individual
                rango = grupos[0]
            
            años_expandidos = expandir_rango_años(rango)
            años_encontrados.extend(años_expandidos)
    
    # Eliminar duplicados y ordenar
    años_encontrados = sorted(list(set(años_encontrados)))
    
    return años_encontrados

In [44]:
import re

def limpiar_token(token):
    # Elimina cualquier caracter que NO sea letra, número o espacio al inicio y final
    return re.sub(r'^[^\w]+|[^\w]+$', '', token)


In [51]:
import re

def parsear_cadena_completa(texto):
    tokens = re.findall(r'\w[\w.-]*|\S', texto)  # separamos palabras y signos

    resultados = []
    i = 0
    marca_actual = None
    permitir_modelo = False  # se activa solo después de una marca o coma

    while i < len(tokens):
        token = tokens[i]

        # Si encontramos una nueva marca, reiniciamos estado y activamos permiso de modelo
        if token in marcas:
            marca_actual = token
            permitir_modelo = True
            i += 1
            continue

        # Si encontramos coma, permitimos modelo (de la marca actual)
        if token == ',' or token =='.':
            permitir_modelo = True
            i += 1
            continue

        # Si está permitido capturar un modelo (tras marca o coma)
        if permitir_modelo and marca_actual:
            modelo = token
            # Verificar modelo compuesto
            if i + 1 < len(tokens) and tokens[i + 1] not in [',']:
                posible_compuesto = f"{modelo} {tokens[i + 1]}"
                if posible_compuesto.lower() in modelos_compuestos_set:
                    modelo = posible_compuesto
                    i += 1

            # Validar modelo
            if (modelo.lower() not in [pb.lower() for pb in palabras_basura]
                and not modelo.replace('.', '').isdigit()
                and modelo not in marcas
                and not re.fullmatch(r'\W', modelo)):
                resultados.append({
                    'marca': marca_actual,
                    'modelo': modelo
                })

            permitir_modelo = False  # solo se permite un modelo por marca o coma
            i += 1
            continue

        # Si no estamos en contexto válido, simplemente avanzamos
        i += 1

    # Extraer cilindrada
    cilindrada = None
    match_dec = re.search(r'(\d\.\d)\s*(?:[LlCc]|L)?', texto)
    match_cc = re.search(r'(\d{3,4})\s*(?:CC|cc|Cc|cC)', texto)

    if match_dec:
        cilindrada = f"{float(match_dec.group(1)):.1f}"
    elif match_cc:
        cc_value = int(match_cc.group(1))
        cilindrada = f"{(cc_value / 1000):.1f}"

    # Extraer años
    años_expandidos = extraer_y_expandir_años(texto)

    # Combinar resultados
    final = []
    for r in resultados:
        if años_expandidos:
            for a in años_expandidos:
                final.append({
                    'marca': r['marca'],
                    'modelo': r['modelo'],
                    'cilindrada': cilindrada,
                    'año': float(a)
                })
        else:
            final.append({
                'marca': r['marca'],
                'modelo': r['modelo'],
                'cilindrada': cilindrada,
                'año': None
            })

    return final


In [52]:
filas_expandidas = []

for _, row in df.iterrows():
    texto = str(row["Nombre"])
    autos = parsear_cadena_completa(texto)

    for auto in autos:
        filas_expandidas.append({
            'Codigo': row["Codigo"],
            'Nombre': row["Nombre"],
            'marca': auto['marca'],
            'modelo': auto['modelo'],
            'cilindrada': auto['cilindrada'],
            'año': auto['año']
        })

df_autos = pd.DataFrame(filas_expandidas)


In [61]:
df_autos[df_autos['marca'] == 'Kia']

,Codigo,Nombre,marca,modelo,cilindrada,año
16,T181-4409-3,Cadena Distribucion Hyundai Accent Rb 1.4 Gran...,Kia,Rio,1.4,NaN
18,T181-4410-7,Cadena Distribucion Hyundai i10 (i-10) Kia Mor...,Kia,Morning,None,NaN
26,T181-4416-6,"Cadena Distribucion Superior Hyundai Santa Fe,...",Kia,Carnival,None,NaN
27,T181-4416-6,"Cadena Distribucion Superior Hyundai Santa Fe,...",Kia,Sorento,None,NaN
28,T181-4416-6,"Cadena Distribucion Superior Hyundai Santa Fe,...",Kia,Sportage,None,NaN
43,T181-5174-K,Cadena de Distribucion 1 Hyundai Grand i10 (i-...,Kia,Morning,1.2,2016.0
44,T181-5174-K,Cadena de Distribucion 1 Hyundai Grand i10 (i-...,Kia,Morning,1.2,2017.0
45,T181-5174-K,Cadena de Distribucion 1 Hyundai Grand i10 (i-...,Kia,Morning,1.2,2018.0
65,T181-7418-9,Cadena Superior Distribucion Hyundai Accent Rb...,Kia,Carens,1.7,2011.0


In [64]:
def limpiar_y_filtrar_resultados(df):
    """
    Elimina registros incompletos SOLO cuando existe otro registro 
    con la misma marca, modelo y año que tenga información completa.
    """
    # 1. Correcciones específicas de modelos
    df['modelo'] = df['modelo'].str.replace(r'^\.', '', regex=True)
    df['modelo'] = df['modelo'].str.replace(r',$', '', regex=True)
    
    correcciones = {
        '.Rio': 'Rio',
        'Carnival,': 'Carnival',
        'Santa': 'Santa Fe',
        'Sonata,': 'Sonata',
        'Tcuson': 'Tucson'
    }
    df['modelo'] = df['modelo'].replace(correcciones)
    
    # 2. Eliminar modelos no válidos
    modelos_invalidos = ['Larga', 'Corta', '46']
    df = df[~df['modelo'].isin(modelos_invalidos)]
    
    # 3. Identificar registros completos (tienen cilindrada Y año)
    df['completo'] = df['cilindrada'].notna() & df['año'].notna()
    
    # 4. Para cada grupo (marca, modelo, año), determinar si existe versión completa
    grupos = df.groupby(['marca', 'modelo', 'año'])
    
    # Lista para almacenar índices a eliminar
    indices_a_eliminar = []
    
    for nombre_grupo, grupo in grupos:
        # Si hay registros completos en este grupo
        if grupo['completo'].any():
            # Marcar los incompletos para eliminar
            indices_a_eliminar.extend(
                grupo[~grupo['completo']].index.tolist()
            )
    
    # 5. Eliminar solo los registros marcados
    df = df.drop(indices_a_eliminar)
    
    # 6. Eliminar columna temporal
    df = df.drop(columns=['completo'])
    
    # 7. Eliminar filas donde marca o modelo son NA
    df = df.dropna(subset=['marca', 'modelo'], how='any')
    
    return df.reset_index(drop=True)


filas_expandidas = []

for _, row in df.iterrows():
    texto = str(row["Nombre"])
    autos = parsear_cadena_completa(texto)

    for auto in autos:
        filas_expandidas.append({
            'marca': auto['marca'],
            'modelo': auto['modelo'],
            'cilindrada': auto['cilindrada'],
            'año': auto['año']
        })

df_autos = pd.DataFrame(filas_expandidas)

# 2. Luego aplicas la limpieza final
df_final = limpiar_y_filtrar_resultados(df_autos)
df_final = df_final.drop_duplicates(subset=['marca', 'modelo', 'año'], keep='first')
df_final.reset_index(drop=True, inplace=True)
df_final['marca'] = df_final['marca'].str.upper()
df_final['modelo'] = df_final['modelo'].str.upper()


In [65]:
df_final

,marca,modelo,cilindrada,año
0,TOYOTA,COROLLA,1.6,2005.0
1,TOYOTA,COROLLA,1.6,2006.0
2,TOYOTA,COROLLA,1.6,2007.0
3,TOYOTA,COROLLA,1.6,2008.0
4,TOYOTA,COROLLA,1.6,2009.0
...,...,...,...,...
63,SSANGYONG,NEW ACTYON,2.0,2018.0
64,HYUNDAI,ACCENT,1.6,2006.0
65,HYUNDAI,ACCENT,1.6,2007.0
66,HYUNDAI,ACCENT,1.6,2008.0


In [67]:
df_final.to_csv("data_limpia/datos_limpios_F1.csv", index=False, encoding='utf-8-sig')